# Proyecto 2
|      Nombre     | Carnet |
|:---------------:|:------:|
| André Rodríguez |  18332 |
|   Cesar Rodas   |  16776 |
|  Javier Ramirez |  18099 |
|  Abril Palencia |  18198 |

### Importación de librerías
Se importan las librerías necesarias para el funcionamiento del laboratorio:

In [133]:
import pandas as pd
import numpy as np
import nltk
import re

nltk.download('stopwords')

from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\cesarvin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Se carga el dataset original desde el archivo train.csv

In [134]:
dataset = pd.read_csv('train.csv')
dataset[:5]

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


Se muestra el tamaño del dataset original.

In [135]:
print("Columnas: " + str(dataset.shape[1]))
print("Filas: " + str(dataset.shape[0]))

Columnas: 5
Filas: 7613


Se muestra el tipo de columnas en el dataset original.

In [136]:
display(dataset.dtypes)

id           int64
keyword     object
location    object
text        object
target       int64
dtype: object

 - Se elimina la columna de id para el texto original, ya que esta no servirá para el análisis
 - Se convierten las columnas de keyword, location y text a tipo 'string' en caso que surga algún error
 - Se cambian los valores NaN por la palabra "None" en las columnas keyword y location para evitar problemas en el futuro

In [137]:
if 'id' in dataset.columns: del dataset['id']

dataset['keyword'].fillna('None',inplace=True)
dataset['location'].fillna('None',inplace=True)
dataset['text'].fillna('None',inplace=True)

dataset['keyword'] = dataset['keyword'].astype(str)
dataset['location'] = dataset['location'].astype(str)
dataset['text'] = dataset['text'].astype(str)

dataset[:15]

,keyword,location,text,target
0,None,None,Our Deeds are the Reason of this #earthquake M...,1
1,None,None,Forest fire near La Ronge Sask. Canada,1
2,None,None,All residents asked to 'shelter in place' are ...,1
3,None,None,"13,000 people receive #wildfires evacuation or...",1
4,None,None,Just got sent this photo from Ruby #Alaska as ...,1
5,None,None,#RockyFire Update => California Hwy. 20 closed...,1
6,None,None,#flood #disaster Heavy rain causes flash flood...,1
7,None,None,I'm on top of the hill and I can see a fire in...,1
8,None,None,There's an emergency evacuation happening now ...,1
9,None,None,I'm afraid that the tornado is coming to our a...,1


Se muestran nuevamente las columnas del dataset luego del procesamiento.

In [138]:
display(dataset.dtypes)

keyword     object
location    object
text        object
target       int64
dtype: object

Eliminación de stopwords en la columna de text. Para ello, se convierten todas las palabras en la oracion a minuscula y luego se elimnan las stopwords.

In [139]:
# funcion para limpiar archivos
def clean_txt(cadena):
    return_text = ''
    
    # removiendo links
    cadena = re.sub('http\S+', '', cadena.strip())
    
    # removiendo espacios html
    cadena = re.sub('%20', ' ', cadena.strip())

    # remover caracteres
    for i, caracter in enumerate(cadena):
        # convierte a minusculas
        caracter = caracter.lower()
        try:
          # remueve cualquier caracter que no sea una letra minuscula o espacio, incluyendo numeros
          if ord(caracter) != 32 and (ord(caracter) < 97 or ord(caracter) > 122) :
              return_text = return_text + ''
          else:
              return_text = return_text + caracter
        except: 
            return_text = return_text + ''

    
    return return_text.strip()

In [140]:
# funcion para remover stepwords
def remove_stepwords(str):
    word_tokens = str.split()
    clean_str = ''

    for word in word_tokens:
        if word not in stopwords_list:
            if clean_str == '':
              clean_str += word
            else:
              clean_str += ' ' + word
    
    return clean_str

In [141]:
stopwords.words('english')
stopwords_list = stopwords.words()

dataset_array = dataset.to_numpy()

for i in range(len(dataset_array)):
    dataset_array[i][0] = clean_txt(dataset_array[i][0])
    dataset_array[i][1] = clean_txt(dataset_array[i][1])
    dataset_array[i][2] = remove_stepwords(clean_txt(dataset_array[i][2]))

clean_dataset = pd.DataFrame(dataset_array,columns=['keyword','location','text','target'])

clean_dataset[:15]

,keyword,location,text,target
0,none,none,deeds reason earthquake may allah forgive us,1
1,none,none,forest fire near ronge sask canada,1
2,none,none,residents asked shelter place notified officer...,1
3,none,none,people receive wildfires evacuation orders cal...,1
4,none,none,got sent photo ruby alaska smoke wildfires pou...,1
5,none,none,rockyfire update california hwy closed directi...,1
6,none,none,flood disaster heavy rain causes flash floodin...,1
7,none,none,top hill see fire woods,1
8,none,none,theres emergency evacuation happening building...,1
9,none,none,afraid tornado coming area,1


Se eliminan los tweets que tienen texto en blanco

In [142]:
clean_dataset = clean_dataset.loc[clean_dataset['text'] != '']

Se eliminan los tweets que no son objetivo para el estudio de desastres

In [143]:
clean_dataset = clean_dataset.loc[clean_dataset['target'] == 1]

Guardado del archivo en un archivo nuevo llamado clean_data.csv para el análisis exploratorio.

In [144]:
clean_dataset.to_csv('clean_data.csv',index=False)